In [3]:
import math
import logging
FORMAT = '[%(name)s:%(levelname)s]  %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT)
logger = logging.getLogger('dbg')

def dprint(s):
    logger.debug(s)

def iprint(s):
    logger.info(s)

logger.setLevel(logging.INFO)

## Algorithm Design Strategies

| Approach | Description |
| ----------- | ----------- |
| Divide & Conquer | Breaks problem into subproblems like original but smaller, solve recursively and combine |
| Dynamic Programming | Applies like _divide & conquer_ but when subproblems overlap making combination more efficient |
| Greedy Algorithms | Problem is divided in such as way that combination is performed in a locally optimal manner |
| Others | Brute-Force, Backtracking, Branch & Bound, Transform & Conquer, etc. |

### Divide & Conquer

- **Divide** the problem into one or more subproblems that are smaller instances of the
same problem.
- **Conquer** the subproblems by solving them recursively.
- **Combine** the subproblem solutions to form a solution to the original problem.

Examples: 
- _Sorting_ - MergeSort, QuickSort, Heapsort
- _Search_ - Binary Search
- _Computation_ - Strassen's Matrix Multiplication
- _Signal Processing_ - FFT
- _Geometric Algorithms_ - Closest Pair

#### Binary Search

_Given a sorted array, find the location of a specific element, or show that the element does not exist_

Split the array down the middle and search recursively:

In [4]:
## search S for element k
def binary_search(S, k, p = None, r = None):
    ## set default sub-array to full-span
    # p is bottom, r is top : [p:r]
    if p == None:
        p = 0
    if r == None:
        r = len(S) - 1
    # midpoint 
    q = (p+r)//2 # floor division
    if p > r:
        return -1 # Not found
    elif S[q] == k:
        # match
        return q
    elif k < S[q]:
        return binary_search(S, k, p, q-1) # Left sub array
    else:
        return binary_search(S, k, q+1, r) # Right sub array

A = [1,2,4,8,16,32,64,128]
print(A)
print(f"{8} is at {binary_search(A, 8)}")
print(f"{9} is at {binary_search(A, 9)}")



[1, 2, 4, 8, 16, 32, 64, 128]
8 is at 3
9 is at -1


### 1D Closest Pair (Using Sort O(nlogn))

The 2D case is more complicated and non-examinable

In [8]:
def mergesort_merge(S, p, q, r):
    ## bit of slicing trickery
    ## extract the two sorted sublists were trying to merge
    L = [S[p]] if p == q else S[p:q+1] 
    R = [S[q+1]] if q+1 == r else S[q+1:r+1]
    # trick to simplify the merge
    # if we append both with infinity, if one list ends 
    # the other will be appended without any logic change 
    L.append(float("inf"))
    R.append(float("inf"))
    i = 0 # L index
    j = 0 # R index
    # k is the index in the top level list we're replacing
    for k in range(p, r+1):
        if L[i] <= R[j]:
            S[k] = L[i]
            i += 1
        else:
            S[k] = R[j]
            j += 1
    


def mergesort(S, p = None, r = None):
    if p == None:
        iprint(S)
        p = 0
    if r == None:
        r = len(S)-1
    if p < r:
        # S is the list, p-r is the sub-array index to sort
        q = math.floor((p+r)/2) # midpoint
        # sort these two sub-arrays
        mergesort(S, p, q)
        mergesort(S, q + 1, r)
        mergesort_merge(S, p, q, r)


def closest_point_lin(S):
    mergesort(S)
    min_distance = float("inf")
    index = -1
    for i in range(1, len(S)):
        delta = S[i] - S[i-1]
        if delta < min_distance:
            min_distance = delta
            index = i
    return([S[index-1], S[index]], min_distance)

A = [1,12,34,26,7,29,55,102,234,222,65]
print(closest_point_lin(A))


[dbg:INFO]  [1, 12, 34, 26, 7, 29, 55, 102, 234, 222, 65]


([26, 29], 3)


### Dynamic Programming

Problems exhibit **optimal sub-structure** and **overlapping subproblems**.

Optimal sub-structure is when optimal solutions to a problem incorporate optimal solutions to related subproblems, which you may solve independently. Dynamic programming builds an optimal solution to the problem from optimal  solutions to subproblems.

**Key Steps:**
1. Characterize the structure of an optimal solution
2. Recursively define the value of an optimal solution
3. Compute the value of an optimal solution (bottom up)
4. Construct an optimal solution from computed information

Examples: Bellman equation, Bellman-Ford, 0-1 Knapsack Problem, Longest common subsequence

#### 0-1 Knapsack Problem

A thief robbing a store wants to take the most valuable load that can be carried in a knapsack capable of carrying
at most $W$ pounds of loot. The thief can choose to take any subset of $n$ items in the store. The $i$ th item is worth $v_i$ dollars and weighs $w_i$ pounds, where $v_i$ and $w_i$ are integers. Which items should the thief take? 

This is called the  0-1 knapsack problem because for each item, the thief must either take it or leave it behind.

In [41]:
W = 10
w = [1,3,4,8,1]
v = [2,4,1,6,6]

# optimal is 0,3,4 to 14 & w = 10

## Recursive Solution O(2^n)

def knapsack_recursive_track(v,w,W,r = None):
    if r == None:
        r = 0
    
    #print(f"Check {r} w:{w[r]} v:{v[r]}")

    if r < len(w) and W > 0:
        # Case 1 - take item at position r if possible
        # find best case with remaining options
        x, s = knapsack_recursive_track(v, w, W-w[r], r + 1)
        x += v[r]
        # Case 2 - DONT take item at position r
        # find best case with remaining options
        y, t = knapsack_recursive_track(v, w, W, r + 1)

        ## pick x but only if we still have a mass budget
        if x > y and W-w[r] >= 0:
            s.append(r)
            return (x, s)
        else:
            return (y, t)
    return 0, []

def knapsack_recursive(v,w,W,r = None):
    if r == None:
        r = len(w)-1
    
    #print(f"Check {r} w:{w[r]} v:{v[r]}")
    #print(r, W)
    if r >= 0 and W > 0:
        # Case 1 - take item at position r
        # find best case with remaining options
        x = knapsack_recursive(v, w, W-w[r], r - 1) + v[r]
        # Case 2 - DONT take item at position r
        # find best case with remaining options
        y = knapsack_recursive(v, w, W, r - 1)

        if x > y and W-w[r] >= 0:
            #print(x)
            return x
        else:
            #print(y)
            return y
    #print(f"Overflow {r} {W}")
    return 0

print(f"Optimal Value: {knapsack_recursive(v,w,W)}")
print(f"Optimal Value: {knapsack_recursive_track(v,w,W)}")

Optimal Value: 14
Optimal Value: (14, [4, 3, 0])


#### Dynamic Programming Soln

![image](media/dynam01.png)

In [76]:
W = 10
w = [1,3,4,8,1]
v = [2,4,1,6,6]

def dynamic_knapsack(v, w, W):
    S = [[0 for x in range(W+1)] for y in range(len(w))]
    # first index is i which is between 0 and n-1
    # second index is weight which is between 1 and W

    for i in range(0, len(w)):
        for weight in range(0, W+1):
            if w[i] <= weight:
                S[i][weight] = max( S[i-1][weight], S[i-1][weight-w[i]] + v[i])
            else:
                 S[i][weight] = S[i-1][weight]
    return S[len(w)-1][W], S

a, Q= dynamic_knapsack(v,w,W)

W = Q[0]
print(f"Weights:  {[x for x in range(len(W))]}")
for i in range(len(Q)):
    print(f"n: {0}->{i} - {Q[i]}")

Weights:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
n: 0->0 - [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
n: 0->1 - [0, 2, 2, 4, 6, 6, 6, 6, 6, 6, 6]
n: 0->2 - [0, 2, 2, 4, 6, 6, 6, 6, 7, 7, 7]
n: 0->3 - [0, 2, 2, 4, 6, 6, 6, 6, 7, 8, 8]
n: 0->4 - [0, 6, 8, 8, 10, 12, 12, 12, 12, 13, 14]
